In [155]:
import cv2
import math
import numpy as np
import pandas as pd

import sys

from PIL import Image

from scipy import fftpack
from scipy.fft import fft2, ifft2
from scipy.linalg import norm
from scipy.signal import deconvolve, fftconvolve
from scipy.optimize import minimize, Bounds

import plotly.express as px

In [156]:
### Blurs
def psf_gaussian(dim: tuple, s1: float = 0, s2: float = 0, i: int = 0, j: int = 0) -> np.ndarray:
    '''
    Returns a point spread function using a Gaussian blur.

    Parameters:
        dim (tuple): dimensions of the image to produce, ex. (3,3) for a 3x3 kernel
        s1 (float): scaling factor for dim[0]
        s2 (float): scaling factor for dim[1]
        i (int): offset for center of psf in dim[0]
        j (int): offset for center of psf in dim[1]

    Returns:
        Normalized matrix for Gaussian blur.
    '''

    # verify that kern size is not zero or negative
    if dim[0] < 1 or dim[1] < 1:
        raise ValueError("Dimensions must be greater than or equal to 1.")

    # verify that i and j offsets are in range [0, dim - 1]
    if i < 0 or i >= dim[0] - 1:
        raise ValueError("i must be in range of [0, dim - 1]")
    if j < 0 or j >= dim[1] - 1:
        raise ValueError("j must be in range of [0, dim - 1]")

    # create matrix of zeros
    p = np.zeros(dim)

    # apply blur
    for x in range(0, dim[0]):
        for y in range(0, dim[1]):
            p[x, y] = math.exp(-0.5 * ((x - i)/s1)**2 - 0.5 * ((y - j)/s2)**2)

    # normalize p values to [0, 1]
    # p = p / np.linalg.norm(p)

    return p

def add_noise(mat: np.ndarray, noise: float) -> np.ndarray:
    '''
    Adds noise to an image.

    Parameters:
        mat (ndarray): matrix to add noise to
        noise (float): amount of noise to add in range [0, 1]

    Returns:
        New matrix with noise added.
    '''
    noisy_mat = mat.copy() + noise * np.random.random(mat.shape)

    # normalize
    norm = np.linalg.norm(noisy_mat)
    # prevent divide by zero errors
    if norm != 0:
        noisy_mat = noisy_mat / np.linalg.norm(noisy_mat)

    return noisy_mat

In [157]:
### Borders
def pad_psf(mat: np.ndarray, size: tuple) -> np.ndarray:
    '''
    Pads a point spread funciton with zeros. The original PSF will
    appear in the upper-lerft corner in order to preserve center
    of PSF for future computations.

    Parameters:
        mat (ndarray): matrix to pad
        size (tuple): desired dimensions of the padded matrix

    Returns:
        Matrix containing the padded data.
    '''

    zeros = np.zeros(size)
    zeros[0:mat.shape[0], 0:mat.shape[1]] = mat
    return zeros

def zeros(mat: np.ndarray, size: tuple = None) -> np.ndarray:
    '''
    Returns a matrix with zero boundary conditions.

    Parameters:
        mat (ndarray): Matrix to add boundary conditions to.
        size (tuple): Number of pixels to include in the boundary.
                      Use None to specify all pixels.
    
    Returns:
        New matrix with boundary conditions.
    '''

    mat_zeros = np.zeros(mat.shape)
    zeros_stack = np.vstack((mat_zeros, mat_zeros, mat_zeros))
    mid_stack   = np.vstack((mat_zeros, mat,       mat_zeros))
    result      = np.hstack((zeros_stack, mid_stack, zeros_stack))

    if size:
        return resize(result, size)
    else:
        return result

def periodic(mat: np.ndarray, size: tuple = None) -> np.ndarray:
    '''
    Returns a matrix with periodic boundary conditions.

    Parameters:
        mat (ndarray): Matrix to add boundary conditions to.
        size (tuple): Number of pixels to include in the boundary.
                      Use None to specify all pixels.
    
    Returns:
        New matrix with boundary conditions.
    '''

    stack  = np.vstack((mat, mat, mat))
    result = np.hstack((stack, stack, stack))

    if size:
        return resize(result, size)
    else:
        return result

def reflexive(mat: np.ndarray, size: tuple = None) -> np.ndarray:
    '''
    Returns a matrix with reflexive boundary conditions.

    Parameters:
        mat (ndarray): Matrix to add boundary conditions to.
        size (tuple): Number of pixels to include in the boundary.
                      Use None to specify all pixels.
    
    Returns:
        New matrix with boundary conditions.
    '''

    mat_lr = np.fliplr(mat)
    mat_ud = np.flipud(mat)
    mat_x  = np.fliplr(mat_ud)

    left_stack  = np.vstack((mat_x,  mat_lr, mat_x))
    mid_stack   = np.vstack((mat_ud, mat,    mat_ud))
    right_stack = np.vstack((mat_x,  mat_lr, mat_x))
    result      = np.hstack((left_stack, mid_stack, right_stack))

    if size:
        return resize(result, size)
    else:
        return result

def resize(mat, padding):
    '''
        Returns a matrix with only the padding needed.  
        Returned matrix has the following structure:
        (x+y) (y) (x+y)  
        (x)   mat   (x)  
        (x+y) (y) (x+y)  

        Parameters:
            mat (mat): original matrix
            padding (tuple): how many pixels to pad, ex (5, 5)
        
        Returns:
            matrix with padding applied
    '''

    padding_x, padding_y = padding

    original_pos = mat.shape[0]/3
    left_bound   = int(original_pos - 2*padding_x)
    right_bound  = int(2*original_pos + 2*padding_x)
    bottom_bound = int(original_pos - 2*padding_y)
    top_bound    = int(2*original_pos + 2*padding_y)
    return mat[bottom_bound:top_bound, left_bound:right_bound]

def undo(mat: np.ndarray, size: tuple = None) -> np.ndarray:
    '''
        Extracts the center part of a matrix by "undoing" the boundary condition operations. If a kernel size
        is given, only the given pixels will be removed; otherwise, it will unpack a matrix into a 3x3 proportion
        and return the middle matrix.

        Parameters:
            mat (ndarray): Matrix containing the original image with boundary conditions.
            size (tuple): Dimensions of padding applied, ex. (3, 3)

        Returns:
            Center of matrix.
    '''

    # kernel size given, so slice it
    if size:
        padding_x = size[0]
        padding_y = size[1]
        return mat[padding_x:-padding_x, padding_y:-padding_y]
    # no kernel size, so assume it's a 3x3 construction
    else:
        _, mid_stack, _ = np.hsplit(mat, 3)
        _, mid_mat,   _ = np.vsplit(mid_stack, 3)
        return mid_mat

In [158]:
### FFT and DCT
def circshift(mat, amt):
    return np.roll(np.roll(mat, amt[0], axis=0), amt[1], axis=1)

def naive_blur_fft(X, P, center):
    return blur_fft(X, P, center, error = 0)

def blur_fft(X, P, center, error = 0):
    # compute eigenvalues
    S = fft2(circshift(P, (0 - center[0], 0 - center[1])))
    # compute blurred image
    B = np.real(ifft2(np.multiply(S, fft2(X))))
    # normalize to values in range [0, 1]
    B = (B - np.min(B))/(np.max(B) - np.min(B)) # normalizes
    # check if noise is present
    if error == 0:
        return B
    # create noisy matrix with shape of blurred image
    noise = B + (np.random.rand(*B.shape) * error)
    # normalize result again
    return (noise - np.min(noise))/(np.max(noise) - np.min(noise)) # normalizes

def naive_deblur_fft(B, P, center):
    # compute eigenvalues
    S = fft2(circshift(P, (0 - center[0], 0 - center[1])))
    # fix for eigenvalues equal to zero
    S = np.where(S != math.inf, S, np.finfo(np.float64).eps)
    # compute deblurred image
    X = np.real(ifft2(np.divide(fft2(B), S)))
    return X

def gcv(alpha, s, bhat):
    phi_d = np.divide(1, (np.abs(s)**2 + alpha**2))
    G = np.divide(np.sum(np.abs(bhat * phi_d)**2), np.sum(phi_d)**2)
    return G

def gcv_tik(s, bhat):
    # helps alleviate divide by zero issues
    # float_info.epsilon is the smallest possible floating point number greater than zero
    lb = max(np.amin(np.abs(s)), sys.float_info.epsilon)
    ub = np.amax(np.abs(s))
    initial_guess = [1]
    alpha = minimize(gcv, initial_guess, \
                     method='L-BFGS-B', bounds=Bounds(lb, ub), args=(s, bhat), \
                     options={'eps': 1.08})
    return alpha['x'][0]

def tik_deblur_fft(B, P, center, alpha = 0):
    # compute eigenvalues
    S = fft2(circshift(P, (0 - center[0], 0 - center[1])))
    # fix for eigenvalues equal to zero
    S = np.where(S != 0, S, float('inf'))
    s = S.flatten() # should be col-wise
    # calculate regularizaton parameter
    bhat = fft2(B)
    bhat = bhat.flatten()
    # TODO: graph results of varying alphas, compute error between true image and deblurred image
    # alpha = gcv_tik(s, bhat)
    # compute Tikhonov regularized solution
    s = np.where(s != math.inf, s, np.finfo(np.float64).eps)
    D = s.conj() * s + abs(alpha)**2
    bhat = s.conj() * bhat
    xhat = np.divide(bhat, D)
    xhat = np.reshape(xhat, B.shape)
    x = np.real(ifft2(xhat))
    return x

In [159]:
### Image utilities
def im_float_to_bmp(mat):
    '''
        Converts floating-point values in a matrix to [0, 255].
        Useful for saving the image.
    '''
    return (mat * 255 / np.max(mat)).astype('uint8')

def load_img(path):
    '''
        Returns a matrix from a given image
    '''
    mat = np.asarray(Image.open(path))
    return mat / 255

def save_img(mat, path, kern_size = None):
    '''
        Saves an image to the specified path. If a kernel size is given,
        it will undo the boundary conditions before saving. Otherwise, it will unpack
        it assuming a full 3x3 deconstruction.
    '''
    Image.fromarray(im_float_to_bmp(undo(mat, kern_size))).save(path)

def save_img_raw(mat, path):
    '''
        Saves an image to the specified path. Assumes no undoing of boundary conditions.
    '''
    Image.fromarray(im_float_to_bmp(mat)).save(path)

In [160]:
psf_size   = (32, 32)             # dimensions of PSF
psf_fn     = psf_gaussian         # specifies function to generate PSF
s          = 2                    # spread of PSF; s = s1 = s2
length     = 3
angle      = 0
noise      = 0.00                 # amount of noise to add to blurred image
border_fn  = periodic             # specifies boundary conditions
blur_fn    = blur_fft             # specifies how to produce a blur
deblur_fn  = naive_deblur_fft       # specifies how to deconvolute a blur
img_name   = 'shapes'             # image to use
sample_img = 'samples/{}.png'.format(img_name)
psf_center = tuple(int(i/2) for i in psf_size)

In [161]:
# load true image from path, values are normalized to [0, 1]
true_image = load_img(sample_img)

# apply boundary conditions
boundary_image = border_fn(true_image, tuple(i * 2 for i in psf_size))
save_img_raw(boundary_image, 'results/1_{}_boundary.png'.format(img_name))

# create the PSF
psf = psf_fn(psf_size, s, s, psf_center[0], psf_center[1])
# psf = blur(psf_fn, psf_size, length, angle, psf_center[0], psf_center[1])
save_img_raw(psf, 'results/2_{}_psf.png'.format(img_name))

# pad PSF to the size of the image
psf_padded = pad_psf(psf, boundary_image.shape)

# blur the true image
blurred_true_image = blur_fn(boundary_image, psf_padded, psf_center)
save_img_raw(blurred_true_image, 'results/3_{}_true_blurred.png'.format(img_name))

# add noise to the blurred true image
blurred_true_image = add_noise(blurred_true_image, noise)
save_img_raw(blurred_true_image, 'results/4_{}_noisy_blurred.png'.format(img_name))

In [162]:
# deblur using true parameters

# TODO: no noise, naive version
#       - add noise, then tik solver (without GCV, do loop for several lambdas logspace(10**-4, 0, 10)), pick best lambda
#       - if that works, then add GCV to compute best lambda automatically
#       - add elements back in

deblurred_image = deblur_fn(blurred_true_image, psf_padded, psf_center)
save_img_raw(deblurred_image, 'results/5_{}_deblurred.png'.format(img_name))

In [163]:
# deblur using alternating minimizer

minimized_true_image = blurred_true_image
minimized_s          = 3
errors               = []
tolerance            = 0.25

def check_convergence(elements, tolerance):
    if len(elements) < 2:
        return False
    return abs(elements[-1] - elements[-2]) < tolerance

# fun(x, *args) -> float
def minimize_blur(values, B, X):
    s             = values[0]
    psf           = psf_fn(psf_size, s, s, psf_center[0], psf_center[1])
    psf_padded    = pad_psf(psf, X.shape)
    Ax            = blur_fn(boundary_image, psf_padded, psf_center)
    return norm(Ax - B, ord='fro')

def minimize_blur_tik(values, B, X):
    s             = values[0]
    psf           = psf_fn(psf_size, s, s, psf_center[0], psf_center[1])
    psf_padded    = pad_psf(psf, X.shape)
    true_im_guess = tik_deblur_fft(B, psf_padded, psf_center)
    return norm(true_im_guess - X, ord='fro')

run_number = 0
for i in range(10):
    print("-- Run {}".format(run_number))
    run_number += 1

     # create PSF from minimized s
    minimized_psf = psf_fn(psf_size, minimized_s, minimized_s, psf_center[0], psf_center[1])
    psf_padded    = pad_psf(psf, blurred_true_image.shape)

    # deconvolve image using minimized s
    minimized_true_image = deblur_fn(blurred_true_image, psf_padded, psf_center)

    # attempt to minimize s 
    minimize_s_result = minimize(minimize_blur_tik, [minimized_s], (blurred_true_image, minimized_true_image))
    # assign minimized value from result to minimized_s
    minimized_s = minimize_s_result['x'][0]

    # compute error between true image and blurred true image
    errors.append(norm(minimized_true_image - boundary_image, ord='fro'))
    print("min s:", minimized_s)

-- Run 0
min s: 1.9899987167756028
-- Run 1
min s: 1.9899987167756028
-- Run 2
min s: 1.9899987167756028
-- Run 3
min s: 1.9899987167756028
-- Run 4
min s: 1.9899987167756028
-- Run 5
min s: 1.9899987167756028
-- Run 6
min s: 1.9899987167756028
-- Run 7
min s: 1.9899987167756028
-- Run 8
min s: 1.9899987167756028
-- Run 9
min s: 1.9899987167756028


In [164]:
# plot errors with different values of s

def get_error(s):
    psf           = psf_fn(psf_size, s, s, psf_center[0], psf_center[1])
    psf_padded    = pad_psf(psf, boundary_image.shape)
    Ax            = blur_fn(boundary_image, psf_padded, psf_center)
    B             = blurred_true_image
    return norm(Ax - B, ord='fro')

s_values = np.linspace(0.1, 16, 100)
df = pd.DataFrame(dict(
    s = s_values,
    errors = [get_error(s) for s in s_values]
))

fig = px.line(df, x = 's', y = 'errors', title = 'Minimizing Error')
fig.update_layout(
    xaxis_title = 's-value in Gaussian blur',
    yaxis_title = 'Error'
)
# f